# Piero Pettenà - RL project
Using lecture jupyter notebooks and adapting them to my project. Credit goes to     

NOTE: maybe 'transition' and 'rewards' methods can be joint.

## Plotting utilities

In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

plt.rcParams['figure.figsize'] = [10, 7]
plt.rcParams['figure.dpi'] = 100 
plt.rcParams['font.size'] = 6

def plot_world(World):
    # ------------------
    Ly, Lx = World.shape

    fig, ax = plt.subplots()
    im = ax.imshow(World, cmap=plt.get_cmap("Spectral"))
    
    # We want to show all ticks...
    ax.set_xticks(np.arange(Lx))
    ax.set_yticks(np.arange(Ly))

    goal = np.where(np.logical_or( World.grid > 0.0, World < -1.0))
    blocks = np.where(World == -1.0)
    # Loop over data dimensions and create text annotations.
    for i in range(Lx):
        for j in range(Ly):
            if np.logical_and(goal[0]==j,goal[1]==i).any():
                text = ax.text(i,j, 'G{}'.format(int(World[j,i])), ha="center", va="center", color="black")
            elif np.logical_and(blocks[0]==j,blocks[1]==i).any():
                 text = ax.text(i,j, 'X', ha="center", va="center", color="black", backgroundcolor="black")
            else:
                pass
    plt.show()
    # -------------------


def plot_world_values(World, Values):
    # ------------------
    Ly, Lx = World.shape

    fig, (ax, ax2) = plt.subplots(1,2)
    im = ax.imshow(World, cmap=plt.get_cmap("Spectral"))

    # We want to show all ticks...
    ax.set_xticks(np.arange(Lx))
    ax.set_yticks(np.arange(Ly))

    goal = np.where(np.logical_or( World > 0.0, World < -1.0))
    blocks = np.where(World == -1.0)
    # Loop over data dimensions and create text annotations.
    for i in range(Lx):
        for j in range(Ly):
            if np.logical_and(goal[0]==j,goal[1]==i).any():
                text = ax.text(i,j, 'G{}'.format(World[j,i]), ha="center", va="center", color="black")
            elif np.logical_and(blocks[0]==j,blocks[1]==i).any():
                text = ax.text(i,j, 'X', ha="center", va="center", color="black", backgroundcolor="black")
            else:
                pass

    im2 = ax2.imshow(Values, cmap=plt.get_cmap("Spectral"))

    # We want to show all ticks...
    ax2.set_xticks(np.arange(Lx))
    ax2.set_yticks(np.arange(Ly))

    # Loop over data dimensions and create text annotations.
    for i in range(Lx):
        for j in range(Ly):
            if np.logical_and(goal[0]==j, goal[1]==i).any():
                text = ax2.text(i,j, 'G{}'.format(World[j,i]), ha="center", va="center", color="black")
            elif np.logical_and(blocks[0]==j,blocks[1]==i).any():
                text = ax2.text(i,j, 'X', ha="center", va="center", color="black", backgroundcolor="black")
            else:
                text = ax2.text(i, j, '{:.2f}'.format(Values[j, i]), ha="center", va="center", color="black")
                
                
    plt.show()
    # -------------------

def plot_world_values_policy(World, Values, Policy):
    # ------------------
    Ly, Lx = World.shape

    fig, (ax, ax2, ax3) = plt.subplots(1,3)
    im = ax.imshow(World, cmap=plt.get_cmap("Spectral"))

    # We want to show all ticks...
    ax.set_xticks(np.arange(Lx))
    ax.set_yticks(np.arange(Ly))

    goal = np.where(np.logical_or( World > 0.0, World < -1.0))
    blocks = np.where(World == -1.0)
    # Loop over data dimensions and create text annotations.
    for i in range(Lx):
        for j in range(Ly):
            if np.logical_and(goal[0]==j,goal[1]==i).any():
                text = ax.text(i,j, 'G-{}'.format(World[j,i]), ha="center", va="center", color="black")
            elif np.logical_and(blocks[0]==j,blocks[1]==i).any():
                text = ax.text(i,j, 'X', ha="center", va="center", color="black", backgroundcolor="black")
            else:
                pass

    im2 = ax2.imshow(Values, cmap=plt.get_cmap("Spectral"))

    # We want to show all ticks...
    ax2.set_xticks(np.arange(Lx))
    ax2.set_yticks(np.arange(Ly))

    # Loop over data dimensions and create text annotations.
    for i in range(Lx):
        for j in range(Ly):
            if np.logical_and(goal[0]==j, goal[1]==i).any():
                text = ax2.text(i,j, 'G{}'.format(World[j,i]), ha="center", va="center", color="black")
                text = ax3.text(i,j, 'G{}'.format(World[j,i]), ha="center", va="center", color="black")
            elif np.logical_and(blocks[0]==j,blocks[1]==i).any():
                text = ax2.text(i,j, 'X', ha="center", va="center", color="black", backgroundcolor="black")
                text = ax3.text(i,j, 'X', ha="center", va="center", color="black", backgroundcolor="black")
            else:
                text = ax2.text(i, j, '{:.2f}'.format(Values[j, i]), ha="center", va="center", color="black")
    
    im3 = ax3.imshow(Values, cmap=plt.get_cmap("Spectral"))
    X = np.arange(Lx)
    Y = np.arange(Ly)
    U, V = Policy[:,:,1], -Policy[:,:,0]
    q = ax3.quiver(X, Y, U, V, color="black")

    plt.show()
    # -------------------
    

In [2]:
# Gets the distance class of a given distance, given the size of the grid L
# distance 0 -> distance less than L/10
# distance 1 -> distance between L/10 and L/5
# distance 2 -> distance between L/5 and L
def get_dist_class(dist, L):
    if dist < L / 10:
        dist = 0
    elif dist < L / 5:
        dist = 1
    elif dist <= L:
        dist = 2
    else:
        print(f"Error: distance out of bounds: dist={dist}")
        return -1
    return dist

# Gets the time class of a given time, given the budget of time B
# time 0 -> time less than B/4
# time 1 -> time between B/4 and B/2
# time 2 -> time between B/2 and 3*B/4
# time 3 -> time between 3*B/4 and B
def get_time_class(time, B):
    if time < B / 4:
        time = 0
    elif time < B / 2:
        time = 1
    elif time < 3 * B / 4:
        time = 2
    elif time <= B:
        time = 3
    else:
        print(f"Error: time out of bounds: time={time}")
        return -1
    return time

# Creating the world
There will be no "block" cells as they are not needed for our problem.

### Definition of a state
1. Distance of agent from datum
2. Direction of search movement
3. Remaining time to find target

In [3]:
import numpy as np
import math

# TYPICAL (GRID)WORLD


class World:
    """World environment class. It contains the grid, the datum and
    goal positions, info on the current and 'explored grid' which contains
    the last moment each block has been visited.

    NOTE: should add a variable that improves visibility of adjacent cells

    Attributes:
        grid:                   grid of the world. cell is 0 if has not been explored yet. +1 if it has been explored
        goal:                   position of target in grid
        datum:                  position of datum (reference) in grid
        current:                current offset (future implementation)
        actions:                List of action that the agent can take. I want these to be part of the environment
        randomExplorerFlag:     if true, action is picked at random each time
        rem_time:               remaining time to find the target
        budget:                 initial budget of time to find the target (will not be updated)
        states:                 states q values of the table
    """

    def __init__(self, Ly=20, Lx=20, goal=(0, 0), rem_time=1000):
        self.goal = goal
        if self.goal == (0, 0):
            self.goal = (
                np.random.randint(Lx),
                np.random.randint(Ly),
            )  # maybe this should be an array

        self.grid = np.full(
            shape=(Lx, Ly), fill_value=0
        )  # Every cell which has not been explored contains 0

        self.grid[goal] = 0  # Goal cell contains 0
        self.actions = np.array(
            [[1, 0], [-1, 0], [0, 1], [0, -1]]
        )  # Actions = [Up, Down, Right, Left]
        self.randomExplorerFlag = True
        self.datum = (Lx // 2, Ly // 2)
        self.budget = rem_time
        self.rem_time = rem_time
        self.current = (0, 0)  # maybe for addition of current
        self.states = np.zeros(
            shape=(3, 4, 4)
        )  # ***************************************************************************
        # Convert the "shape()" parameter to a variable that we can choose
        # i.e. try to generalize this code for different number of distance classes,
        # directions and time classes

    def plot_world(self, agent=None):
        """Plots the world (copied from tutor)"""
        Ly, Lx = self.grid.shape

        fig, ax = plt.subplots()
        im = ax.imshow(self.grid, cmap=plt.get_cmap("Spectral"))

        # We want to show all ticks...
        ax.set_xticks(np.arange(Lx))
        ax.set_yticks(np.arange(Ly))

        # Loop over data dimensions and create text annotations.
        for i in range(Lx):
            for j in range(Ly):
                if np.logical_and(self.goal[0] == j, self.goal[1] == i):
                    text = ax.text(
                        i,
                        j,
                        "G".format(int(self.grid[j, i])),
                        ha="center",
                        va="center",
                        color="black",
                        backgroundcolor="blue",
                    )
                elif np.logical_and(self.datum[0] == j, self.datum[1] == i):
                    text = ax.text(
                        i,
                        j,
                        "D",
                        ha="center",
                        va="center",
                        color="black",
                        backgroundcolor="green",
                    )
                    pass
                elif (agent != None) and (
                    np.logical_and(agent.pos[0] == j, agent.pos[1] == i)
                ):
                    text = ax.text(
                        i,
                        j,
                        "A".format(int(self.grid[j, i])),
                        ha="center",
                        va="center",
                        color="black",
                        backgroundcolor="white",
                    )
        plt.show()

    def plot_world_with_path(self, agent=None):
        """Plots the world (copied from tutor)"""
        Ly, Lx = self.grid.shape

        fig, ax = plt.subplots()
        im = ax.imshow(self.grid, cmap=plt.get_cmap("Spectral"))

        # We want to show all ticks...
        ax.set_xticks(np.arange(Lx))
        ax.set_yticks(np.arange(Ly))

        # Loop over data dimensions and create text annotations.
        for i in range(Lx):
            for j in range(Ly):
                if np.logical_and(self.goal[0] == j, self.goal[1] == i):
                    text = ax.text(
                        i,
                        j,
                        "G".format(int(self.grid[j, i])),
                        ha="center",
                        va="center",
                        color="black",
                        backgroundcolor="blue",
                    )
                elif np.logical_and(self.datum[0] == j, self.datum[1] == i):
                    text = ax.text(
                        i,
                        j,
                        "D",
                        ha="center",
                        va="center",
                        color="black",
                        backgroundcolor="green",
                    )

        if agent != None:
            for cell in agent.explored:
                text = ax.text(
                    cell[1],
                    cell[0],
                    ".".format(int(self.grid[cell[1], cell[0]])),
                    ha="center",
                    va="center",
                    color="white",
                    backgroundcolor="white",
                )

        text = ax.text(
            i,
            j,
            "A".format(int(self.grid[agent.pos[1], agent.pos[0]])),
            ha="center",
            va="center",
            color="black",
            backgroundcolor="white",
        )

        plt.show()

    def plot_world_policy(self, actions):
        policy = [
            [1, 0] if x == 0 else [-1, 0] if x == 1 else [0, 1] if x == 2 else [0, -1]
            for x in actions
        ]

        Ly, Lx = self.grid.shape
        fig, ax = plt.subplots()
        im = ax.imshow(self.grid, cmap=plt.get_cmap("Spectral"))
        X = np.arange(Lx)
        Y = np.arange(Ly)
        U, V = policy[:, :, 1], -policy[:, :, 0]
        q = ax3.quiver(X, Y, U, V, color="black")

        plt.show()

    def discretize_state(self, ddist, rem_time):
        dist = get_dist_class(dist=ddist, L=self.grid.shape[0])
        time = get_time_class(time=rem_time, B=self.budget)

        return dist, time
       
    # Returns the tuple of state information. State is defined on agent position, therefore it needs
    # to receive the agen as a parameter.
    def give_state(self, agent):
        print(f"Agent position: {agent.pos}")
        ddist = math.dist(agent.pos, self.datum)
        dir = agent.dir 
        # Discretize distance and time:
        dist, time = self.discretize_state(ddist, self.rem_time)

        return agent.pos[0], agent.pos[1], dir, dist, time

In [15]:
class Agent:
    """Agent = explorator

    Attributes:
        pos:            current position of agent
        visibility:     how far the agent can see (1 means only adjacent cells)
        env:
        action_value:   State-Action value matrix
        ddist:          distance from datum
        ??choices:        last choices for each position
        ##explored:       matrix with already explored cells -> already in environment
        dir:            direction of movement (=last action)
        state:          state q values of the table
    """

    def __init__(self, world: World):
        self.pos = world.datum  # initial position is the same as the datum
        self.visibility = 1  # try to change visibility of agent (sees not only strictly adjacent cells)
        self.env = world  # environment of agent. Should be a reference, not a full copy
        # self.action_value = np.zeros((world.grid.shape, len(world.actions)))
        self.ddist = 0  # distance from datum
        # self.explored = np.zeros_like(world.grid)  # matrix with already explored cells
        self.dir = [0, 0]

        # Defining some constants
        self.n_actions = self.env.actions.shape[0]
        self.Ly, self.Lx = self.env.grid.shape
        self.state = np.zeros(
            (world.grid.shape[0], world.grid.shape[1], len(world.actions), 3, 4)
        )  # where 3 is the number of distance classes and 4 the number of time classes

    # Returns the 4 nearby cells to visit. This could be extended to diagonal movement.
    def get_nearby_cells(self):
        selfpos = np.array(self.pos)
        top = tuple(selfpos + [-1, 0])
        bottom = tuple(selfpos + [1, 0])
        right = tuple(selfpos + [0, 1])
        left = tuple(selfpos + [0, -1])
        nearby = [bottom, top, left, right]

        return nearby


    def remove_outside_cells(self, coordinates):
        """
        Remove coordinates outside of a grid of size Lx by Ly.

        Parameters:
        - coordinates: numpy array of shape (n, 2) representing 2D integer coordinates.
        - Lx: int, size of the grid along the x-axis.
        - Ly: int, size of the grid along the y-axis.

        Returns:
        - filtered_coordinates: numpy array with valid coordinates within the grid.
        """
        # Check if the input is a numpy array
        if not isinstance(coordinates, np.ndarray):
            raise ValueError("Input 'coordinates' must be a NumPy array.")

        # Check if the shape of the array is (n, 2)
        if len(coordinates.shape) != 2 or coordinates.shape[1] != 2:
            raise ValueError("Input 'coordinates' must be a 2D array with shape (n, 2).")

        # Create a boolean mask for valid coordinates
        valid_mask = (
            (0 <= coordinates[:, 0])
            & (coordinates[:, 0] < self.Lx)
            & (0 <= coordinates[:, 1])
            & (coordinates[:, 1] < self.Ly)
        )

        # Use boolean indexing to filter the coordinates array
        filtered_coordinates = coordinates[valid_mask]

        return filtered_coordinates

    # Function that chooses an action for the agent. It can be random or based on the action-value matrix.
    # Validity of the action is checked before it is returned.
    # It has to be inserted in a while loop which doesn't update self.env.rem_time, otherwise remaining time
    # will decrease even when an action is not taken but found to be invalid.
    # This is bad especially if the agent can see more than one cell at a time, as it is increasingly
    # more likely that it runs into already visited cells (which are considered not valid actions).
    def chooseAction(self):
        if self.env.randomExplorerFlag:
            # Pick random action
            nearby = np.array(self.get_nearby_cells())
            # border check:
            nearby = self.remove_outside_cells(nearby)
            print(f"nearby  = {nearby}")

            not_visited_nearby = nearby.copy()

            # Removing already visited cells
            for i, cell in enumerate(not_visited_nearby):
                row, col = cell
                if self.env.grid[row, col] == 1:
                    not_visited_nearby = np.delete(not_visited_nearby, i)

            while len(not_visited_nearby) != 0:
                rand_action = np.random.randint(len(not_visited_nearby))
                S_new = not_visited_nearby[rand_action]
                direction = tuple(S_new - np.array(self.pos))
                print("type of S_new = ", type(S_new))
                print(f"S_new = {S_new}, self.pos = {self.pos}")
                print(f"Returning direction {direction}")

                return direction

            if len(not_visited_nearby) == 0:
                print(
                    "All nearby cells have been visited. Going to pick a random action."
                )
                S_new = nearby[np.random.randint(len(nearby))]
                print(f"S_new = {S_new}")
                direction = tuple(S_new - np.array(self.pos))
                return direction

        else:
            print("Needs implementation for randomExplorerFlag = False")
        return direction

    # Function that returns the reward of the system. If the agent is in the goal position, it returns 10.
    # It has to be called whenever an action is taken, as it only checks the current position of the agent.
    # Here we could implement the possibility of viewing nearby cells.
    def reward(self):
        if not np.array_equal(self.pos, self.env.goal):
            return -1
        else:
            return 10
        # idea: reward is also influenced by the distance
        # reward = reward - self.ddist

    # Function that chooses an action using "chooseAction" and updates the position of the agent.
    # It only updates the position if the action is not [0,0].
    def update_position(self):
        action = [0, 0]
        while np.array_equal(action, [0, 0]):
            action = self.chooseAction()
        self.pos = tuple(np.array(self.pos) + np.array(action))
        print(f"chooseAction got to position {self.pos}")

        if np.array(self.pos).any() < 0:
            print(f"Error: position out of bounds: pos={self.pos}")
            return -1

    def update_state(self, reward):
        """Updates the state of the agent"""
        state_index = self.env.give_state(agent=self)
        self.state[state_index] = reward

    def transition(self):
        """Chooses a valid action, updates the position of the agent and the remaining time."""
        self.update_position()
        # get the reward for the new state
        reward = self.reward()
        self.update_state(reward)

        # update visited cells
        self.env.grid[self.pos] = 1
        # update remaining time in environment attribute
        self.env.rem_time -= 1
        print(f"remaining time = {self.env.rem_time}")

In [16]:
Lx = 20
Ly = 20

world = World(Lx=Lx, Ly=Ly, rem_time=300)
ag = Agent(world)

print(ag.state.shape)
while(world.rem_time > 0):
    ag.transition()

(20, 20, 4, 3, 4)
nearby  = [[11 10]
 [ 9 10]
 [10  9]
 [10 11]]
type of S_new =  <class 'numpy.ndarray'>
S_new = [11 10], self.pos = (10, 10)
Returning direction (1, 0)
chooseAction got to position (11, 10)
Agent position: (11, 10)
remaining time = 299
nearby  = [[12 10]
 [10 10]
 [11  9]
 [11 11]]
type of S_new =  <class 'numpy.ndarray'>
S_new = [10 10], self.pos = (11, 10)
Returning direction (-1, 0)
chooseAction got to position (10, 10)
Agent position: (10, 10)
remaining time = 298
nearby  = [[11 10]
 [ 9 10]
 [10  9]
 [10 11]]
type of S_new =  <class 'numpy.int64'>
S_new = 9, self.pos = (10, 10)
Returning direction (-1, -1)
chooseAction got to position (9, 9)
Agent position: (9, 9)
remaining time = 297
nearby  = [[10  9]
 [ 8  9]
 [ 9  8]
 [ 9 10]]
type of S_new =  <class 'numpy.ndarray'>
S_new = [ 9 10], self.pos = (9, 9)
Returning direction (0, 1)
chooseAction got to position (9, 10)
Agent position: (9, 10)
remaining time = 296
nearby  = [[10 10]
 [ 8 10]
 [ 9  9]
 [ 9 11]]
type

In [ ]:
array = np.array([[10, 11], [8, 11], [9, 10], [9, 12]])
print(f"array = {array}")
print(f"type of array = {type(array)}")

In [ ]:
array[array[:,1] > 11]

# Playground

In [ ]:
Lx = 10
Ly = 10
n_runs = 1000

weights = np.zeros(n_runs)
action_mtx = np.zeros((Lx, Ly))

for i in range(n_runs):
    goal = (np.random.randint(Lx), np.random.randint(Ly))
    world = World(Ly, Lx, goal)
    ag = Agent(world)
    weights[i] = ag.single_episode()

    for row in range(Lx):
        for col in range(Ly):
            action_mtx[row, col] = action_mtx[row, col] + ag.choices[row, col]*weights[i]
            #action_mtx[row, col] = action_mtx[row, col] + ag.choices[row, col]


action_mtx = action_mtx/(np.sum(weights))
#action_mtx = action_mtx/(n_runs)

print("ag.choices = ", ag.choices)

print("weights = ", weights)
print("action matrix: ")
print(action_mtx)
#world.plot_world_policy(ag.choices)

print("Visual representation of the gridworld:")
world.plot_world_with_path(ag)

# Transition and Rewards

 - Transitions: Given a state S and the Action A, we return the new state, taking care that we do not do forbidden actions. We will use deterministic transitions.

 - Rewards: Given a state S and the Action A and the new state S', we also have the probability to receive a reward R.

__PS: Achtung!__
The convention for python arrays is $A[i_y, i_x]$, where $i_y$ is the row-index and $i_x$ is the column-index... So the convention with _up_, _down_, _right_ and _left_ directions consistent, but may be a bit confusing: Use special care!

In [ ]:
# The list of actions I can take: Actions = [Up, Down, Right, Left]
Actions = np.array([[1,0],[-1,0],[0,1],[0,-1]])

def transition(S, A, world):
    """
    Takes the current position S and selected action A,
    and returns the resulting new S given a world World.
    """
    # I try to move
    S_new = S + A
    Ly, Lx = world.grid.shape
    # if I go out of the world, I stay still
    if ((S_new[0] == Ly) or np.any(S_new == -1) or (S_new[1] == Lx)):
        S_new = S
    # if I found a block I stay still
    elif world.grid[S_new[0],S_new[1]] == -1:
        S_new = S
    # returns the new state
    return S_new 

def rewards(S, A, S_new, world):
    """
    Takes the current position S and selected action A,
    and returns the resulting reward given that it has ended up in S_new and 
    the gridworld is World.
    """
    # reward is always zero...
    reward = 0
    # expect when I reach the final goal
    if (world.grid[S_new[0], S_new[1]] > 0) or (world.grid[S_new[0], S_new[1]] < -1):
        reward = world.grid[S_new[0], S_new[1]]
    # return the reward
    return reward


In [ ]:
def update_values(Values, world, gamma, p=0.9):
    """
    Takes the current matrix of values (V_k(s) )
    The associated gridworld,
    And computes the bellman operator   
    V_(k+1) (s) = max_a { sum_s'r   p(r, s'| s, a)(r + gamma V_k(s') }
    And the relative best policy
    pi_(k+1)(s) = argmax_a { sum_s'r   p(r, s'| s, a)(r + gamma V_k(s') }
    
    In the case with deterministic (p=1) or stochastic (p<1) actions.
    Returns V_(k+1)(s) and pi_(k+1)(s)
    """
    
    # -----------------------------------------------------------
    # The dimension of the world
    Ly, Lx = world.shape
    # initialize the vectors to store the new values and policy
    NewValues = np.zeros((Ly,Lx))
    NewPolicy = np.zeros((Ly,Lx,2))
    # 
    goal = np.where(np.logical_or( world.grid > 0.0, world.grid < -1.0))
    
    # --------------- UPDATE -------------------------------------
    # cycle over all the states
    for ix in range(Lx):
        for iy in range(Ly):
            # state is defined by its indices
            S = np.array([iy, ix])
            
            # skip blocked squares
            if world.grid[S[0],S[1]] != -1:
                maxvalue = -100
                
                # it "tries out" all actions and store the best
                for A in Actions:
                    new_S = transition(S, A, world.grid)
                    R = rewards(S, A, new_S, world.grid)

                    value_action = 0.0
                    
                    value_action += R + gamma*Values[new_S[0], new_S[1]]
                    
                    if (value_action > maxvalue):
                        maxvalue = value_action
                        bestact = A
                    # -------------------------
                    # Q: What happens in a tie?
                    # -------------------------

                # It stores the new value and policy of state S
                NewValues[S[0],S[1]] = maxvalue
                NewPolicy[S[0],S[1],:] = bestact
    # --------------------------------------------------------------

    # --------------------------------------------------------------
    for gx, gy in zip(goal[0],goal[1]):
        NewValues[gx, gy] = 0
        NewPolicy[gx, gy] = [0,0]
    # --------------------------------------------------------------
    return NewValues, NewPolicy